In [64]:
import requests
import pandas as pd
import json

# 1. Elerhetö 10-Q es 10-K jelentesek listaja
Megadja, h mely 10-Q es 10-K jelentesek elerhetök a SEC EDGAR-on

In [ ]:
# SEC elvárja, hogy User-Agent legyen megadva (pl. email címmel)
headers = {"User-Agent": "your@emailadress.com"}

In [66]:
# Albemarle CIK (10 karakterre nullákkal feltöltve)
cik = "0000915913"

In [67]:
url = f"https://data.sec.gov/submissions/CIK{cik}.json"
# ez egy JSON file-t ad melyben benne vannak a vallalat fö adatai (iparag, ticker, fiscal year ends, address etc)
# illetve a lista az összes jelenteshez (nem csak 10-Q es 10-K), plusz datum es befogadas timestampje
response = requests.get(url, headers=headers)
data = response.json()

In [68]:
# A 'filings' -> 'recent' tartalmazza a legfrissebbeket
# csak egy lista. jelentesekröl
recent = data['filings']['recent']

In [69]:
# DataFrame-be rendezés
df = pd.DataFrame(recent)

In [70]:
# Csak a 10-K és 10-Q típusok kellenek
df_filings = df[df['form'].isin(['10-K', '10-Q'])]
# filingDate = a cég által jelzett benyújtás dátuma.
# acceptanceDateTime = amikor a SEC feldolgozta és a dokumentum elérhetővé vált → innentől elmeletileg letölthetö, a timestamp UTC-ben van (Z - Zulu)
df_filings.head()

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,core_type,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
9,0000915913-25-000133,2025-08-04,2025-06-30,2025-08-04T20:19:15.000Z,34,10-Q,001-12658,251180533,,XBRL,12001318,1,1,alb-20250630.htm,10-Q
25,0000915913-25-000084,2025-04-30,2025-03-31,2025-04-30T20:35:44.000Z,34,10-Q,001-12658,25896070,,XBRL,10249597,1,1,alb-20250331.htm,10-Q
52,0000915913-25-000026,2025-02-12,2024-12-31,2025-02-12T22:09:12.000Z,34,10-K,001-12658,25615946,,XBRL,26036967,1,1,alb-20241231.htm,10-K
72,0000915913-24-000158,2024-11-06,2024-09-30,2024-11-06T21:45:12.000Z,34,10-Q,001-12658,241431760,,XBRL,10915836,1,1,alb-20240930.htm,10-Q
80,0000915913-24-000139,2024-07-31,2024-06-30,2024-07-31T20:41:53.000Z,34,10-Q,001-12658,241162622,,XBRL,9878819,1,1,alb-20240630.htm,10-Q


In [79]:
# Érdekes oszlopok kiválasztása
df_filings = df_filings[['form', 'reportDate', 'filingDate', 'acceptanceDateTime', 'accessionNumber', 'primaryDocument']]

df_filings.tail()


,form,reportDate,filingDate,acceptanceDateTime,accessionNumber,primaryDocument
909,10-Q,2014-03-31,2014-04-24,2014-04-24T21:21:41.000Z,0000915913-14-000021,a0331201410q.htm
938,10-K,2013-12-31,2014-02-25,2014-02-25T22:24:49.000Z,0000915913-14-000008,a1231201310k.htm
970,10-Q,2013-09-30,2013-10-18,2013-10-18T20:38:52.000Z,0000915913-13-000007,a930201310q.htm
974,10-Q,2013-06-30,2013-07-19,2013-07-19T19:24:25.000Z,0001193125-13-295546,d547207d10q.htm
997,10-Q,2013-03-31,2013-04-19,2013-04-19T18:03:07.000Z,0001193125-13-162568,d501631d10q.htm


In [72]:
# SEC dokumentum URL-ek összerakása
base_url = "https://www.sec.gov/Archives/edgar/data"
df_filings['url'] = df_filings.apply(
    lambda row: f"{base_url}/{int(cik)}/{row['accessionNumber'].replace('-', '')}/{row['primaryDocument']}",
    axis=1
)

df_filings.tail()

,form,reportDate,filingDate,acceptanceDateTime,accessionNumber,primaryDocument,url
909,10-Q,2014-03-31,2014-04-24,2014-04-24T21:21:41.000Z,0000915913-14-000021,a0331201410q.htm,https://www.sec.gov/Archives/edgar/data/915913...
938,10-K,2013-12-31,2014-02-25,2014-02-25T22:24:49.000Z,0000915913-14-000008,a1231201310k.htm,https://www.sec.gov/Archives/edgar/data/915913...
970,10-Q,2013-09-30,2013-10-18,2013-10-18T20:38:52.000Z,0000915913-13-000007,a930201310q.htm,https://www.sec.gov/Archives/edgar/data/915913...
974,10-Q,2013-06-30,2013-07-19,2013-07-19T19:24:25.000Z,0001193125-13-295546,d547207d10q.htm,https://www.sec.gov/Archives/edgar/data/915913...
997,10-Q,2013-03-31,2013-04-19,2013-04-19T18:03:07.000Z,0001193125-13-162568,d501631d10q.htm,https://www.sec.gov/Archives/edgar/data/915913...


# 2. Szamszeru adatok jelentesenkent

#### A JSON-ben van a facts key, ezen belul lehet több kategoria
- dei --> Dokumentum és entitás információk
- us-gaap --> Pénzügyi adatok jelentése
- invest --> Befektetési társaságok speciális adatai

#### Problema lehet az is hogy ha egy ceg US-GAAP kodot valt, akkor a JSON-ban a regi es az uj is szerepel 2 darabbol kell összerakni

In [73]:
url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"

# ez a szimpla response, mindent tartalmaz
response = requests.get(url, headers=headers)
data = response.json()

In [74]:

# teljes eredeti JSON mentése fájlba
with open("albemarle_companyfacts.json", "w") as f:
    json.dump(data, f, indent=2)

print("JSON fájl letöltve: albemarle_companyfacts.json")

JSON fájl letöltve: albemarle_companyfacts.json


In [75]:
gaap_tags = list(data['facts']['us-gaap'].keys())
print("GAAP kódok száma Albemarle eseten:", len(gaap_tags))

GAAP kódok száma Albemarle eseten: 655


#### Nekem hosszu tavon szerintem a facts.us-gaap key alatti tartalom kell

Viszont a problema ugyanaz mint korabban. FAC (Financial Accounting Classification) és az US GAAP fogalmakat össze kell hozni.
Meg kell csinalni egy mapping tablat, ezt nem fogom meguszni.
1. El kell dömnteni mit akarok szamolni es azt miböl
2. vegig kell gyurni a cegeket es megnezni ki mit hasznal a SEC EDGAR jelentesek szamit összenezni a JSON fileokkal

In [81]:
# Csak az us-gaap részt
us_gaap_facts = data['facts'].get('us-gaap', {})

# JSON fájlba mentés
with open("albemarle_usgaap_facts.json", "w") as f:
    json.dump(us_gaap_facts, f, indent=2)

print("US-GAAP JSON fájl letöltve: albemarle_usgaap_facts.json")
print("Lekérhető tag-ek száma:", len(us_gaap_facts))

US-GAAP JSON fájl letöltve: albemarle_usgaap_facts.json
Lekérhető tag-ek száma: 655
